In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

2023-03-27 12:16:59.016475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 12:16:59.151162: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-27 12:16:59.155405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 12:16:59.155430: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
ton_iot = pd.read_csv("/home/lolol/Uni/Bachelor/Data/Train_Test_Network.csv")
ton_iot = ton_iot.sample(frac = 1)
ton_iot.head()


,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
350970,1556331134,192.168.1.195,59238,192.168.1.79,9197,tcp,http,0.009189,132,396,...,0,0,-,-,-,-,-,-,0,normal
378711,1556341034,192.168.1.195,53759,192.168.1.152,1880,tcp,http,13.400623,936,464,...,0,0,-,-,-,-,-,-,0,normal
410216,1556423414,192.168.1.195,49667,239.255.255.250,1900,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
278602,1556260863,132.255.172.18,65413,192.168.1.194,80,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
242457,1556249023,192.168.1.180,42901,192.168.1.190,53,udp,dns,0.006207,40,318,...,0,0,-,-,-,-,-,-,0,normal


In [3]:
ton_iot = ton_iot.drop(["ts", "src_ip", "dst_ip", "src_port", "dst_port", "dns_query"], axis=1)
ton_iot_features = ton_iot.copy()
ton_iot_labels = ton_iot_features.pop('label')


In [4]:
inputs = {}

for name, column in ton_iot_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'proto': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'proto')>,
 'service': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'service')>,
 'duration': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'duration')>,
 'src_bytes': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'src_bytes')>,
 'dst_bytes': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dst_bytes')>,
 'conn_state': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'conn_state')>,
 'missed_bytes': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'missed_bytes')>,
 'src_pkts': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'src_pkts')>,
 'src_ip_bytes': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'src_ip_bytes')>,
 'dst_pkts': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dst_pkts')>,
 'dst_ip_bytes': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dst_ip_bytes')>,


In [5]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(ton_iot[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs


2023-03-27 12:17:02.211661: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-27 12:17:02.211705: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-27 12:17:02.211725: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lolol-Yoga-7-15ITL5): /proc/driver/nvidia/version does not exist
2023-03-27 12:17:02.211976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<KerasTensor: shape=(None, 14) dtype=float32 (created by layer 'normalization')>

In [6]:
preprocessed_inputs = [all_numeric_inputs]


In [18]:
for name, input in inputs.items():
    if input.dtype == tf.float32:
        continue

    lookup = layers.StringLookup(vocabulary=np.unique(ton_iot_features[name]))
    one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x)


KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), name='string_lookup_48/Identity:0', description="created by layer 'string_lookup_48'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name=None), name='category_encoding_48/bincount/DenseBincount:0', description="created by layer 'category_encoding_48'")

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), name='string_lookup_49/Identity:0', description="created by layer 'string_lookup_49'")
KerasTensor(type_spec=TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), name='category_encoding_49/bincount/DenseBincount:0', description="created by layer 'category_encoding_49'")

KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.int64, name=None), name='string_lookup_50/Identity:0', description="created by layer 'string_lookup_50'")
KerasTensor(type_spec=TensorSpec(shape=(None, 14), dtype=tf.float32, name=None), name='category_encoding_50/bincount/DenseBi

In [8]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

ton_iot_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)


In [9]:
ton_iot_features_dict = {name: np.array(value) 
                         for name, value in ton_iot_features.items()}

ton_iot_train = {name: np.array(value) for name, value in ton_iot_features[:200000].items()}
ton_iot_test = {name: np.array(value) for name, value in ton_iot_features[200000:].items()}

labels_train = ton_iot_labels[:200000]
labels_test = ton_iot_labels[200000:]

In [10]:
features_dict = {name:values[:1] for name, values in ton_iot_features_dict.items()}
ton_iot_preprocessing(features_dict)


<tf.Tensor: shape=(1, 271), dtype=float32, numpy=
array([[-0.01846602, -0.01044403, -0.00978122, -0.0044646 , -0.00420183,
        -0.00858199, -0.00428797, -0.00837676, -0.06553465, -0.20334838,
        -0.1618931 , -0.00460319, -0.00313824, -0.02231446,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
 

In [22]:
def ton_iot_model(preprocessing_head, inputs):
    body = tf.keras.Sequential([
        layers.Embedding(output_dim=100, input_dim=100000, mask_zero=True),
        layers.Bidirectional(layers.LSTM(50, return_sequences=True)),
        layers.Dropout(0.2),
        layers.Bidirectional(layers.LSTM(50, activation="relu", return_sequences=False)),
        layers.Dropout(0.2),
        layers.Dense(50, activation="linear"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="softmax")
    ])

    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)

    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

ton_iot_model = ton_iot_model(ton_iot_preprocessing, inputs)


In [23]:
ton_iot_model.fit(x=ton_iot_train, y=labels_train, epochs=3)


Epoch 1/3
2766/6250 [============>.................] - ETA: 16:16 - loss: 0.6505 - accuracy: 0.3490

KeyboardInterrupt: 

In [ ]:
score = ton_iot_model.evaluate(ton_iot_test, labels_test, verbose=1)
